In [1]:
# Impordid
import os
import csv
from datasets import load_dataset
from src.transformers import BertTokenizer, BertForMaskedLM, BertConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
input_folder = "korpus"
input_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename[-3:] == "tsv"]
input_files

['korpus\\estonian_nc17.vert.19_lyh.tsv', 'korpus\\test.tsv']

In [3]:
%%time
train_dataset = load_dataset("csv", data_files={'train': input_files[1]}, names = ["text"], delimiter = "\t")["train"]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: total: 62.5 ms
Wall time: 610 ms


In [4]:
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt", max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")

In [50]:
# Võtab iga batchi (tekstid korpusest, kus laused on eraldatud \n sümboliga, 'text' : lause1 \n lause2 \n lause3)
# Tokenieerib laused ja lisab batchile labelid
# Väjund (x tähistab kõikide lausete arvu loetud batchi tekstides)
# {'input_ids': x*128*2 tensor, 'attention_mask': x*128 tensor, 'token_type_ids': x*128 tensor, 'labels': x*128*2 tensor}
def tokeniseeri_batch(batch):
    batch_laused = [lause for para in batch["text"] for lause in para.split("\n")]
    tokeniseeritud = tokenizer(batch_laused, max_length = 128, padding = "max_length", truncation = True, return_tensors = "pt")
    tokeniseeritud["labels"] = tokeniseeritud["input_ids"]
    return tokeniseeritud

In [52]:
%%time
train_dataset_sisend = train_dataset.map(tokeniseeri_batch, batched=True, remove_columns=["text"])
train_dataset_sisend.set_format(type ='torch')

  0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: total: 578 ms
Wall time: 584 ms


In [ ]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

In [ ]:
config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)

training_args = TrainingArguments(
    output_dir='./train_results',
    per_device_train_batch_size=32,
    max_steps=10,
    learning_rate=1e-4,
    logging_dir='./train_logs',
    logging_steps=5, 
    save_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator = data_collator,
    train_dataset=train_dataset_sisend
)

trainer.train()

In [8]:
model = BertForMaskedLM.from_pretrained("train_results\checkpoint-10")

In [54]:
import torch
from tqdm import tqdm
dataloader_test = torch.utils.data.DataLoader(train_dataset_sisend, batch_size = 16)
loop = tqdm(dataloader_test, leave = True)
for batch in loop:
    input_ids = batch["input_ids"]
    output = model(input_ids)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.74s/it]


torch.Size([14, 128, 111])